In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_df=pd.read_csv('../input/covidctscan3d/train_df.csv')
train_df.label=train_df.label.map({0:'non-covid',1:'covid'})
val_df=pd.read_csv('../input/covidctscan3d/val_df.csv')
val_df.label=val_df.label.map({0:'non-covid',1:'covid'})
test_df=pd.read_csv('../input/covidctscan3d/test_df.csv')
test_df.label=test_df.label.map({0:'non-covid',1:'covid'})

In [3]:
train_df.head()

,image,label
0,../input/covidctscan3d/train/non-covid/ct_scan...,non-covid
1,../input/covidctscan3d/train/covid/ct_scan_206...,covid
2,../input/covidctscan3d/train/covid/ct_scan_95/...,covid
3,../input/covidctscan3d/train/covid/ct_scan_579...,covid
4,../input/covidctscan3d/train/covid/ct_scan_185...,covid


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
   )

In [5]:
img_shape=224
batch_size=100

In [6]:
train_generator=data_gen.flow_from_dataframe(train_df,directory='',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="image",
                                                      y_col='label',
                                                      class_mode='binary',
                                                      shuffle=True,
                                                      batch_size=batch_size)

val_generator=data_gen.flow_from_dataframe(val_df,directory='',
                                                    target_size=(img_shape,img_shape),
                                                    x_col="image",
                                                    y_col='label',
                                                    class_mode='binary',
                                                    shuffle=False,
                                                    batch_size=batch_size)
test_generator=data_gen.flow_from_dataframe(test_df,directory='',
                                                    target_size=(img_shape,img_shape),
                                                    x_col="image",
                                                    y_col='label',
                                                    class_mode='binary',
                                                    shuffle=False,
                                                    batch_size=batch_size)

Found 270244 validated image filenames belonging to 2 classes.
Found 65428 validated image filenames belonging to 2 classes.
Found 75532 validated image filenames belonging to 2 classes.


In [7]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers
from tensorflow.keras.models import Model
model =DenseNet121(weights ='imagenet', include_top=False, input_shape = (img_shape,img_shape,3))
x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
predictions = layers.Dense(1, activation="sigmoid")(x)
model = Model(inputs=model.input, outputs=predictions)

29089792/29084464 [==============================] - 4s 0us/step


In [8]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [9]:
results = model.fit(train_generator,epochs=3,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              #callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=2, min_lr=0.000001)]
                   )

Epoch 1/3
2702/2702 [==============================] - 7189s 3s/step - loss: 0.4719 - accuracy: 0.7699 - val_loss: 0.5474 - val_accuracy: 0.7389
Epoch 2/3
2702/2702 [==============================] - 5515s 2s/step - loss: 0.2791 - accuracy: 0.8786 - val_loss: 0.8720 - val_accuracy: 0.7603
Epoch 3/3
2702/2702 [==============================] - 5603s 2s/step - loss: 0.1766 - accuracy: 0.9288 - val_loss: 0.8219 - val_accuracy: 0.7529


In [10]:
model.save("densenet.h5")

In [11]:
model.evaluate(val_generator)

655/655 [==============================] - 1040s 2s/step - loss: 0.8239 - accuracy: 0.7533


[0.8239209651947021, 0.7533318996429443]